<a href="https://colab.research.google.com/github/Nomaqui/ENTREGAS/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leer el contenido, y cargar las librerias que vamos a usar


In [7]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.read_csv('train.csv')

# Guardar columna objetivo y ID
y = df['RENDIMIENTO_GLOBAL']

X = df.drop(columns=['RENDIMIENTO_GLOBAL', 'ID'])


Separar columnas por tipo

In [8]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categóricas:", cat_cols)
print("Numéricas:", num_cols)

Categóricas: ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']
Numéricas: ['PERIODO', 'coef_1', 'coef_2', 'coef_3', 'coef_4']


Imputar numéricas con la media

In [9]:
imp_num = SimpleImputer(strategy='mean')
X[num_cols] = imp_num.fit_transform(X[num_cols])

Imputar categóricas con la moda

In [10]:
imp_cat = SimpleImputer(strategy='most_frequent')
X[cat_cols] = imp_cat.fit_transform(X[cat_cols])

Mostrar la tabla con los datos imputados

Convertir columnas en one-hot

In [11]:
X_convertida = pd.get_dummies(X, columns=cat_cols, drop_first=True)

Mostrar la tabla con las columnas debidas en one-hot

In [12]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test= train_test_split(X_convertida, y,  test_size=0.01, random_state=19, stratify=y)
clf = RandomForestClassifier(
    n_estimators=400,
    class_weight='balanced',
    # Más árboles suelen mejorar el rendimiento
    max_depth=None,           # Limita profundidad para evitar overfitting
    min_samples_split=5,      # Requiere más muestras para dividir
    min_samples_leaf=2,       # Reduce overfitting
    max_features='sqrt',      # Usa solo raíz de variables en cada split
    random_state=19
)
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', min_samples_leaf=2,
                       min_samples_split=5, n_estimators=400, random_state=19)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        alto       0.53      0.62      0.57      8781
        bajo       0.44      0.57      0.50      8649
  medio-alto       0.32      0.24      0.28      8581
  medio-bajo       0.33      0.25      0.29      8614

    accuracy                           0.42     34625
   macro avg       0.41      0.42      0.41     34625
weighted avg       0.41      0.42      0.41     34625

[[5481  884 1539  877]
 [ 829 4919 1139 1762]
 [2618 2073 2082 1808]
 [1449 3232 1750 2183]]


In [14]:
df_test = pd.read_csv("test.csv")
ids_test = df_test["ID"]
X_test = df_test.drop(columns=["ID"])

In [15]:
# Imputar valores faltantes
X_test[num_cols] = imp_num.transform(X_test[num_cols])
X_test[cat_cols] = imp_cat.transform(X_test[cat_cols])

# Codificación one-hot con las mismas columnas que el entrenamiento
X_test_convertida = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# Asegurar que tenga las mismas columnas que X_convertida
X_test_convertida = X_test_convertida.reindex(columns=X_convertida.columns, fill_value=0)

In [16]:
y_test_pred = clf.predict(X_test_convertida)

In [17]:
print(len(y_test_pred))

296786


In [18]:
submission = pd.DataFrame({
    "ID": ids_test,
    "RENDIMIENTO_GLOBAL": y_test_pred
})

In [19]:
submission.to_csv("submission.csv", index=False)
